In [ ]:
!git clone https://github.com/PedroOrtix/ID_Inpainter.git
%cd ID_Inpainter
!pip install -r requirements.txt -q
!pip install "jax[cuda12_pip]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html -q

In [ ]:
%cd ID_Inpainter

In [ ]:
from paddleocr import PaddleOCR
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt

from utils import separar_cadenas, mostrar_diccionario_ascii

import ocr
from ocr import recalcular_cuadricula_rotada
from ocr import comparar_imagenes
from ocr import reemplazar_parte_imagen

from run import simple_inpaint

import matplotlib.pyplot as plt
from scipy import ndimage

In [ ]:
#open a file containing the image
img = Image.open('images/imagen_dni.jpg')

# SOLO SI LA IMAGEN TIENE MUCHA RESOLUCIÓN
# Especificar el nuevo tamaño (por ejemplo, la mitad de la resolución original)
# nuevo_tamaño = (img.width // 2, img.height // 2)
# # Redimensionar la imagen
# img = img.resize(nuevo_tamaño, Image.ANTIALIAS)

img_array = np.array(img)

# Inicializar el modelo OCR Paddle
model = PaddleOCR(use_angle_cls=True, lang='es')  # 'es' para español
# Realiza OCR
result = model.ocr(img_array)
bounds = ocr.convert_paddle_to_easyocr(result)

# Imprime los nombres (textos) de los cuadros delimitadores
# tres nombres /n tres nombres
lista_elementos_detectados = []
for bound in bounds:
    lista_elementos_detectados.append(bound[1])
    
dict_elems = separar_cadenas(lista_elementos_detectados)
mostrar_diccionario_ascii(dict_elems)

In [ ]:
img_resized, coordenadas_originales = ocr.recortar_imagen(bounds, "STEVEN", img_array, nueva_dimension=512)

In [ ]:
img_pil = Image.fromarray(img_resized).convert('RGB')
img_pil.save("images/imagen_dni_recortada.jpg")

In [ ]:
# Inicializar el modelo OCR Paddle
model = PaddleOCR(use_angle_cls=True, lang='es')  # 'es' para español
# Step 2: Perform OCR on the resized and cropped image to identify word bounding boxes
bounds_resized = model.ocr(np.array(img_pil))
bounds_resized = ocr.convert_paddle_to_easyocr(bounds_resized)
bounds_resized

In [ ]:
from run import process_image
modified_images, right_bounds, coordenadas_originales = process_image(palabra = "STEVEN", 
                replace = "FRANCISCO",
                bounds=bounds,
                img_array=img_array,
                slider_step=30,
                slider_guidance=2,
                slider_batch=6,
                show_plot=True,
                save_intermediate_images=True)


In [ ]:
# elegir cual te gusta mas
modified_images[5].save("images/modified_image.jpg")

In [ ]:
# ver la comparacion entre lo que se ha hecho inpaint
coodinates = np.array(right_bounds[0][0], dtype = int).flatten()
comparar_imagenes("images/imagen_dni_recortada.jpg", 
                  "images/modified_image.jpg", 
                  coodinates, 
                  save_path=None,
                  adjust_temp=True)


In [ ]:
# reemplazar la parte de la imagen original(512x512) por la modificada (boundong box modificada)
img_recortada_mod = reemplazar_parte_imagen("images/imagen_dni_recortada.jpg",
                                            "images/modified_image.jpg",
                                            coodinates,
                                            adjust_temp=False)

In [ ]:
# superponer la imagen 512x512 modificada a la original
img_array = img_array[:, :, :3]
x_min, y_min, x_max, y_max = coordenadas_originales
img_array[y_min:y_max, x_min:x_max] = img_recortada_mod

In [ ]:
Image.fromarray(img_array).save("images/resultado_final.jpg")